In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

c:\Users\dltjf\anaconda3\envs\tensor\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\dltjf\anaconda3\envs\tensor\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\dltjf\anaconda3\envs\tensor\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\dltjf\anaconda3\envs\tensor\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [1]:
img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)

z_dim = 100

- LeakyReLU 활성화 함수가 좀 더 훈련 결과를 향상시키는 경향이 있음
- 일반 ReLU(0또는 1)와 달리 작은 기울기를 사용하여, 그라디언트가 사라지는 것을 막기 때문
- 핸즈 온 머신러닝 416p: 0.01정도의 기울기를 부여하여, LeakyReLU(z) = max(az, z)로 정의

In [8]:
def build_generator(img_shape, z_dim):
    model = Sequential()

    model.add(Dense(128, input_dim = z_dim))
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    model.add(Reshape(img_shape))
    return model

In [4]:
def build_discriminator(img_shape):
    model = Sequential()
    
    model.add(Flatten(input_shape = img_shape))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [5]:
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [9]:
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer = Adam(), metrics = ['accuracy'])

generator = build_generator(img_shape, z_dim)

discriminator.trainable = False

gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer = Adam())

In [ ]:
def sample_images(generator, image_grid_rows = 4, image_grid_columns = 4, z_dim = 4): # z_dim은 generator에도 설정되어있으므로 일치시킬 필요가 있음
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))
    
    gen_imgs = generator.predict(z)

    gen_imgs = 0.5 * gen_imgs + 0.5 # 이미지 범위를 [0,1] 사이로 스케일 조절, 기존에는 -1~1 사이
    
    fig, axs = plt.subplot(image_grid_rows, image_grid_columns, figsize = (4,4), sharey = True, sharex = True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i ,j].imshow(gen_imgs[cnt, :, :, 0], camp='gray')
            axs[i, j].axis('off')
            cnt += 1

In [12]:
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):

    (X_train, _), (_, _) = mnist.load_data()

    # 정규화 진행. [0,255] -> [-1, 1]
    X_train = X_train / 127.5 - 1.0
    X_train = np.expand_dims(X_train, axis=3) 

    real = np.ones(batch_size, 1)
    
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):
        idx = np.random.randint(0, X_train.shape[0], batch_size) # 최소 0, 최대 X_train.shape[0]에서 batch_size개의 값 추출
        imgs = X_train[idx]

        z = np.random.normal(0, 1, (batch_size, 100))

        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(imgs, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake) # total loss가 이 딥러닝 모델의 loss이므로 해당 수식이 적용됨

        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)

        g_loss = gan.train_on_batch(z, real)

        if (iteration + 1) % sample_interval == 0:
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration+1)

            print("%d [D 손실: %f, 정확도: %.2f%%] [G 손실: %f]" % (iteration+1, d_loss, 100*accuracy, g_loss))

            sample_images(generator)


In [13]:
batch_size = 16

(X_train, _), (_, _) = mnist.load_data()

# 정규화 진행. [0,255] -> [-1, 1]
X_train = X_train / 127.5 - 1.0
X_train = np.expand_dims(X_train, axis=3) 

np.random.randint(0, X_train.shape[0], batch_size)

array([38698, 19910, 13014, 19726, 30683, 11197, 23589,  5793, 13318,
       28265, 25987, 19384, 38661, 28728, 49234, 51052])